In [1]:
import numpy as np
import pandas as pd

In [2]:
df_ctpr = pd.read_csv('datasets/csv_files/city_target_passenger_rating.csv')
df_city = pd.read_csv('datasets/csv_files/dim_city.csv')
df_date = pd.read_csv('datasets/csv_files/dim_date.csv')
df_rtd  = pd.read_csv('datasets/csv_files/dim_repeat_trip_distribution.csv')
df_fps  = pd.read_csv('datasets/csv_files/fact_passenger_summary.csv')
df_ft   = pd.read_csv('datasets/csv_files/fact_trips.csv')
df_mtnp = pd.read_csv('datasets/csv_files/monthly_target_new_passengers.csv')
df_mtt  = pd.read_csv('datasets/csv_files/monthly_target_trips.csv')

# Questions

## Primary

### Q1 - Top and Bottom Performing Cities

In [3]:
city_merge = pd.merge(df_ft, df_city, how='left', on='city_id')
t3 = city_merge.groupby(by='city_name').size().sort_values(ascending=False).head(3).reset_index(name='Total_trips')
b3 = city_merge.groupby(by='city_name').size().sort_values(ascending=False).tail(3).sort_values().reset_index(name='Total_trips')

In [4]:
t3

,city_name,Total_trips
0,Jaipur,76888
1,Lucknow,64299
2,Surat,54843


In [5]:
b3

,city_name,Total_trips
0,Mysore,16238
1,Coimbatore,21104
2,Visakhapatnam,28366


### Q2 - Average Fare per Trip by City

In [6]:
avg_fpt = city_merge.groupby(by='city_name').agg(
    avg_fare=('fare_amount', 'mean'),
    avg_distance_km=('distance_travelled(km)', 'mean')
).sort_values(by='avg_fare',ascending=False)

avg_fpt = avg_fpt.assign(avg_cost_per_km=lambda df:df.avg_fare/df.avg_distance_km).round(2)

avg_fpt

,avg_fare,avg_distance_km,avg_cost_per_km
city_name,,,
Jaipur,483.92,30.02,16.12
Kochi,335.25,24.07,13.93
Chandigarh,283.69,23.52,12.06
Visakhapatnam,282.67,22.55,12.53
Mysore,249.71,16.50,15.14
Indore,179.84,16.50,10.90
Coimbatore,166.98,14.98,11.15
Lucknow,147.18,12.51,11.76
Vadodara,118.57,11.52,10.29


### Q3 - Average Ratings by City and Passenger Type

In [7]:
avg_ratings = city_merge.groupby(by=['city_name','passenger_type']).agg(
    avg_passenger_rating=('passenger_rating', 'mean'),
    avg_driver_rating=('driver_rating', 'mean'),
    no_of_trips=('trip_id', 'count')
).sort_values(by=['passenger_type','avg_passenger_rating',],ascending=[True,False]).reset_index()
avg_ratings

,city_name,passenger_type,avg_passenger_rating,avg_driver_rating,no_of_trips
0,Kochi,new,8.987394,8.985350,26416
1,Jaipur,new,8.985018,8.988246,45856
2,Mysore,new,8.982964,8.982878,11681
3,Visakhapatnam,new,8.976151,8.979995,12747
4,Chandigarh,new,8.489158,7.992120,18908
5,Indore,new,8.485837,7.970800,14863
6,Coimbatore,new,8.485788,7.990604,8514
7,Surat,new,7.984173,6.994925,11626
8,Vadodara,new,7.979263,7.004147,10127
9,Lucknow,new,7.977429,6.990406,16260


### Q4 - Peak and Low Demand Months by City

In [8]:
date_merge = pd.merge(df_ft, df_date, on='date', how='left')
date_city_merge = pd.merge(date_merge, df_city, on='city_id', how='left')
demand_by_month = date_city_merge.groupby(by=['city_name','month_name']).agg(
    no_of_trips=('trip_id','count')
).sort_values(by='city_name',ascending=False).reset_index()

peak_months = demand_by_month.loc[demand_by_month.groupby('city_name')['no_of_trips'].idxmax()]
low_months = demand_by_month.loc[demand_by_month.groupby('city_name')['no_of_trips'].idxmin()]

peak_low_months = pd.concat([
    peak_months.assign(Demand='Peak'),
    low_months.assign(Demand='Low')
]).sort_values(by='city_name').reset_index(drop=True)

peak_low_months

,city_name,month_name,no_of_trips,Demand
0,Chandigarh,February,7387,Peak
1,Chandigarh,April,5566,Low
2,Coimbatore,March,3680,Peak
3,Coimbatore,June,3158,Low
4,Indore,May,7787,Peak
5,Indore,June,6288,Low
6,Jaipur,February,15872,Peak
7,Jaipur,June,9842,Low
8,Kochi,May,10014,Peak
9,Kochi,June,6399,Low


### Q5 - Weekend vs. Weekday Trip Demand by City

In [9]:
weekend_vs_weekday = date_city_merge.groupby(['city_name', 'day_type'])['trip_id'].count().reset_index()

comparison = weekend_vs_weekday.pivot(index='city_name', columns='day_type', values='trip_id').reset_index()

comparison['Preference'] = comparison['Weekday'] - comparison['Weekend']

comparison['Strong Demand'] = comparison['Preference'].apply(
    lambda x: 'Weekday' if x > 0 else ('Weekend' if x < 0 else 'Equal')
)
comparison
# weekend_vs_weekday

day_type,city_name,Weekday,Weekend,Preference,Strong Demand
0,Chandigarh,19914,19067,847,Weekday
1,Coimbatore,12576,8528,4048,Weekday
2,Indore,21198,21258,-60,Weekend
3,Jaipur,32491,44397,-11906,Weekend
4,Kochi,22915,27787,-4872,Weekend
5,Lucknow,49617,14682,34935,Weekday
6,Mysore,6424,9814,-3390,Weekend
7,Surat,37793,17050,20743,Weekday
8,Vadodara,20310,11716,8594,Weekday
9,Visakhapatnam,15100,13266,1834,Weekday


### Q6 - Repeat Passenger Frequency and City Contribution Analysis

In [10]:
rpf = df_rtd.groupby('city_id').agg(
    total_repeat_passenger=('repeat_passenger_count', 'sum')
)
rpf_rtd_merge = pd.merge(rpf, df_rtd, on='city_id', how='left')
rpf_city_merge = pd.merge(rpf_rtd_merge, df_city, on='city_id', how='left')
rpf_city_merge = rpf_city_merge.assign(percent_passenger_for_city=lambda df: (df.repeat_passenger_count/df.total_repeat_passenger)*100)

rpf_city_merge = rpf_city_merge[rpf_city_merge['trip_count'].isin(['8-Trips','9-Trips', '10-Trips'])]

rpf_city_merge = rpf_city_merge.groupby(by=['city_name','trip_count'])[['repeat_passenger_count','percent_passenger_for_city']].sum().sort_values(['repeat_passenger_count'], ascending=[False]).reset_index().round(2)

rpf_city_merge

,city_name,trip_count,repeat_passenger_count,percent_passenger_for_city
0,Lucknow,8-Trips,617,6.43
1,Surat,8-Trips,539,6.24
2,Vadodara,8-Trips,251,5.78
3,Indore,8-Trips,235,3.26
4,Jaipur,8-Trips,184,1.90
5,Lucknow,9-Trips,183,1.91
6,Chandigarh,8-Trips,176,3.47
7,Indore,9-Trips,172,2.38
8,Coimbatore,8-Trips,157,6.15
9,Surat,9-Trips,150,1.74


### Q7 - Monthly Target Achievement Analysis for Key Metrics

In [11]:
performance_data = pd.merge(df_fps, df_mtt, on=['city_id', 'month'], how='left')
performance_data = pd.merge(performance_data, df_mtnp, on=['city_id', 'month'], how='left')
performance_data = pd.merge(performance_data, df_ctpr, on='city_id', how='left')
performance_data = pd.merge(performance_data, df_city, on='city_id', how='left')

performance_data.rename(columns={'month': 'date'}, inplace=True)

performance_data = pd.merge(performance_data, df_date, on='date', how='left')

total_trips = date_city_merge.groupby(['city_id','month_name']).agg(total_trips=('trip_id','count'),avg_passenger_rating=('passenger_rating', 'mean')).reset_index()
performance_data = pd.merge(performance_data, total_trips, on=['city_id','month_name'], how='right').sort_values(by=['city_id'])

In [12]:
performance_data['trips_status'] = performance_data.apply(
    lambda row: 'Exceeded' if row['total_trips'] > row['total_target_trips']
    else 'Met' if row['total_trips'] == row['total_target_trips']
    else 'Missed', axis=1
)

In [13]:
performance_data['trips_percent_diff'] = (
    (performance_data['total_trips'] - performance_data['total_target_trips']) / performance_data['total_target_trips']
) * 100

In [14]:
performance_data['new_passengers_status'] = performance_data.apply(
    lambda row: 'Exceeded' if row['new_passengers'] > row['target_new_passengers']
    else 'Met' if row['new_passengers'] == row['target_new_passengers']
    else 'Missed', axis=1
)

In [15]:
performance_data['new_passengers_percent_diff'] = (
    (performance_data['new_passengers'] - performance_data['target_new_passengers']) / performance_data['target_new_passengers']
) * 100

In [16]:
performance_data['rating_status'] = performance_data.apply(
    lambda row: 'Exceeded' if row['avg_passenger_rating'] > row['target_avg_passenger_rating']
    else 'Met' if row['avg_passenger_rating'] == row['target_avg_passenger_rating']
    else 'Missed', axis=1
)

In [17]:
performance_data['rating_percent_diff'] = (
    (performance_data['avg_passenger_rating'] - performance_data['target_avg_passenger_rating']) / performance_data['target_avg_passenger_rating']
) * 100

In [18]:
performance_data.drop(columns=['date','city_id','repeat_passengers','start_of_month','day_type'],inplace=True)

In [19]:
col1 = performance_data.pop('city_name')
performance_data.insert(0,'city_name',col1)
col2 = performance_data.pop('month_name')
performance_data.insert(1,'month_name',col2)

In [20]:
performance_data.head().round(2)

,city_name,month_name,new_passengers,total_passengers,total_target_trips,target_new_passengers,target_avg_passenger_rating,total_trips,avg_passenger_rating,trips_status,trips_percent_diff,new_passengers_status,new_passengers_percent_diff,rating_status,rating_percent_diff
0,Visakhapatnam,April,1845,2837,5000,2000,8.5,4938,8.37,Missed,-1.24,Missed,-7.75,Missed,-1.52
1,Visakhapatnam,February,2380,3170,4500,2500,8.5,4793,8.47,Exceeded,6.51,Missed,-4.80,Missed,-0.41
2,Visakhapatnam,January,2513,3163,4500,2500,8.5,4468,8.55,Missed,-0.71,Exceeded,0.52,Exceeded,0.63
3,Visakhapatnam,June,1900,2702,5000,2000,8.5,4478,8.40,Missed,-10.44,Missed,-5.00,Missed,-1.15
4,Visakhapatnam,March,2170,3093,4500,2500,8.5,4877,8.43,Exceeded,8.38,Missed,-13.20,Missed,-0.87


In [21]:
tourism_cities = ["Jaipur", "Mysore", "Chandigarh"]
business_cities = ["Lucknow", "Surat", "Vadodara", "Indore"]


performance_data['city_type'] = performance_data['city_name'].apply(
    lambda x: 'Tourism' if x in tourism_cities else ('Business' if x in business_cities else 'Mixed')
)

# Group by city type to analyze patterns
city_type_analysis = performance_data.groupby(['city_type'])[[
    'total_trips', 'new_passengers', 'avg_passenger_rating'
]].sum().reset_index().round(2)

# Calculate aggregate percentage differences by city type
percent_diff_analysis = performance_data.groupby('city_type')[[
    'trips_percent_diff', 'new_passengers_percent_diff', 'rating_percent_diff'
]].mean().reset_index().round(2)


In [22]:
city_type_analysis

,city_type,total_trips,new_passengers,avg_passenger_rating
0,Business,193624,52876,164.10
1,Mixed,100172,47677,149.04
2,Tourism,132107,76445,151.52


In [23]:
percent_diff_analysis

,city_type,trips_percent_diff,new_passengers_percent_diff,rating_percent_diff
0,Business,-7.77,5.38,-8.19
1,Mixed,1.09,2.29,-1.65
2,Tourism,11.77,-8.54,2.00


### Q8 - Highest and Lowest Repeat Passenger Rate (RPR%) by City and Month

### A

In [31]:
df_fps['rpr'] = ((df_fps['repeat_passengers']/df_fps['total_passengers']) * 100).round(2)

In [33]:
rpr_city = df_fps.groupby(['city_id', 'month']).agg(rpr_percent=('rpr','mean')).reset_index()

In [37]:
rpr_city = rpr_city.merge(df_city, on='city_id', how='left')


In [41]:
top_cities = rpr_city.sort_values(by='rpr_percent', ascending=False).head(2)
top_cities

,city_id,month,rpr_percent,city_name_x,city_name_y
16,GJ01,2024-05-01,49.92,Surat,Surat
17,GJ01,2024-06-01,49.17,Surat,Surat


In [42]:
bottom_cities = rpr_city.sort_values(by='rpr_percent', ascending=True).head(2)
bottom_cities

,city_id,month,rpr_percent,city_name_x,city_name_y
25,KA01,2024-02-01,7.99,Mysore,Mysore
24,KA01,2024-01-01,8.08,Mysore,Mysore


### B

In [44]:
rpr_month = df_fps.groupby('month').agg(
    total_repeat_pass=('repeat_passengers','sum'),
    total_passenger=('total_passengers','sum')).reset_index()

In [54]:
rpr_month['rpr_percent'] = (
    rpr_month['total_repeat_pass'] / rpr_month['total_passenger']
) * 100
rpr_month

,month,total_repeat_pass,total_passenger,rpr_percent
0,2024-01-01,8343,44672,18.676128
1,2024-02-01,9523,45724,20.827137
2,2024-03-01,10584,41398,25.566452
3,2024-04-01,11013,37633,29.264210
4,2024-05-01,12167,36349,33.472723
5,2024-06-01,9681,32533,29.757477


In [52]:
highest_rpr_month = rpr_month.nlargest(1, 'rpr_percent')
highest_rpr_month

,month,total_repeat_pass,total_passenger,rpr_percent
4,2024-05-01,12167,36349,33.472723


In [53]:
lowest_rpr_month = rpr_month.nsmallest(1, 'rpr_percent')
lowest_rpr_month

,month,total_repeat_pass,total_passenger,rpr_percent
0,2024-01-01,8343,44672,18.676128
